# INPUT playlist, OUTPUT playlist, 기준 : 장르분포도, 방식ALS

In [ ]:
# !pip install implicit

In [63]:
# 모듈 불러오기
import os
import json
import numpy as np
import pandas as pd
import scipy.sparse as spr
from tqdm.notebook import tqdm

from itertools import chain
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import implicit

In [64]:
# 데이터 불러오기

with open('data/train.json',encoding='utf-8-sig') as f:
    train_dict = json.load(f)
    
with open('data/song_meta.json',encoding='utf-8-sig') as f:
    song_dict = json.load(f)
    
with open('data/genre_gn_all.json',encoding='utf-8-sig') as f:
    genre_dict = json.load(f)
    
train = pd.DataFrame.from_dict(train_dict)
genre_gn_all = pd.read_json('data/genre_gn_all.json', typ = 'series') # 장르 읽어오기
song = pd.DataFrame.from_dict(song_dict)
genre = pd.DataFrame(genre_gn_all, columns = ['gnr_name']).reset_index().rename(columns = {'index' : 'gnr_code'}) # 장르코드 : gnr_code, 장르명 : gnr_name 



df_train = train.copy()
df_song = song.copy()

# song data 사용할 컬럼만 가져오기
df_gnr_id_song = song[['song_gn_dtl_gnr_basket','id']].copy()
# 컬럼 알아보기 쉽게 한글로 바꾸기
df_gnr_id_song = df_gnr_id_song.rename(columns={'song_gn_dtl_gnr_basket':'장르','id':'곡id'})



# df_train[장르, 장르id, 장르cnt] columns 추가 (json파일 만들기)
df_train_with_gnr_cnt.json 만들기

In [65]:
from itertools import chain
genre_code_list = list(genre_dict.keys())
genre_id_dict = dict(zip(genre_code_list,range(len(genre_code_list)))) # {code : code_id}

# 노래 : 장르 딕셔너리 만들기 # {song_id : [gnr_id,gnr_id2...]}
songid_genre_dict = dict(zip(df_gnr_id_song['곡id'].tolist(),df_gnr_id_song['장르'].tolist()))

# df_train['장르']
song_id_lists = df_train['songs'].tolist() # playlist의 [[song_id, song_id2 ...],[ ...]]
song_genre_lists = [] 

for song_id_list in song_id_lists:  
    temp = [] 
    for song_id in song_id_list: 
        temp.append(songid_genre_dict[song_id]) 
    song_genre_lists.append(list(chain.from_iterable(temp))) # 이중리스트를 플랫하게 만듦.
    
df_train['장르'] = song_genre_lists # playlist의 [[gnr, gnr2...],[ ... ]]
# df_train.head(2)

# df_train[장르id]
df_train['장르id'] = df_train['장르'].map(lambda x : [ genre_id_dict[i] for i in x]) 
# df_train.head(2)

# df_train['장르cnt']
df_train['장르cnt'] = df_train['장르'].apply(lambda x : len(x))
# df_train.head(2)

# 장르cnt가 0인 것들 임의의 값 주기 # 나중에 전체 나눗셈을 할 때 분모가 0일 경우 Null값이 생기기 때문
df_train['장르cnt'].replace(0,1, inplace=True)
# df_train[df_train['장르cnt'] == 0] # 잘 없어졌는지 확인.


# json파일로 만들기
# df_train.to_json('data/df_train_with_gnr_cnt.json') # 파일 생성

---

# df_train DF sparse matrix 만들기 (npz 저장하기)

In [66]:
from collections import defaultdict, Counter

my_dict = defaultdict(dict)
genre_id_lists = df_train['장르id'].tolist() # genre_id_lists   # 이중 리스트 [[111,110,57 ....]]

# sparse matrix를 만들기 위한 row, col, dat 데이터 뽑아오기
for i in range(len(df_train)):
    my_dict[i] = dict(Counter(genre_id_lists[i]))
row =[] # playlist
col =[] # genre
dat =[] # gnr_cnt

for k, v in my_dict.items():
    for vk,vv in v.items():
        row.append(k)
        col.append(vk)
        dat.append(vv)

# print(row)
# print(col)
# print(dat)


spr_ply_gnr = spr.csr_matrix((dat, (row, col)), shape=(len(df_train), len(genre)))
spr_ply_gnr  # row = ply, col = genre, dat = gnr_cnt

# row별로 장르cnt를 나눠주면 각 곡의 비율이 나오니까 노래의 개수로 인한 데이터 편중이 일어나지 않을 것.

spr_ply_gnr_pct = spr_ply_gnr / df_train[['장르cnt']].values  # spr_ply_gnr_pct = matrix 형태


spr_spr_ply_gnr_pct= spr.csr_matrix(spr_ply_gnr_pct) # sparse matrix 형태로 저장

# npz 저장하기
# spr.save_npz('data/spr_spr_ply_gnr_pct.npz', spr_spr_ply_gnr_pct) 

---

# 2개 playlist의 코사인 유사도 구하기
playlist와 playlist 간의

> 유사한 장르 분포를 가진 ply(user) 간을 비교하기에 앞서서 `장르 cnt의 비율`로 만든 이유는     
장르 개수의 격차가 클 때 코사인 유사도를 구하면
(예를 들어 비슷한 장르 분포를 가졌지만 A(ply)에 포함된 곡의 장르가 1000개, B(ply)에 포함된 곡의 장르가 10개라고 할 때)    
서로 비슷하지 않다는 결과가 도출되기 때문에 `(A) ply의 각 장르 / (A) ply total cnt`로 나눠서 장르의 총합이 1이 되도록 `비중(%)`으로 구함.         
멘토님은 `countervectorizer nomalization`의 의견을 주셨음. 유튜브에 찾아봐야 할 듯     

In [81]:
# npz 불러오기 
spr_spr_ply_gnr_pct = spr.load_npz('data/spr_spr_ply_gnr_pct.npz') # csr형태

spr_ply_gnr_pct = spr_spr_ply_gnr_pct.todense() # matrix 형태

from sklearn.metrics.pairwise import cosine_similarity

# cosine_similarity(spr_ply_gnr_pct[1],spr_ply_gnr_pct[3]) # matrix 형태

cosine_similarity(spr_spr_ply_gnr_pct[1],spr_spr_ply_gnr_pct[4]) # csr 형태

array([[0.04929709]])

# als_model 피클 저장(따로 ipynb 만들어놓음 - 10.27일자)
> spr_spr_ply_gnr_pct.npz 가져와서 만듦

In [84]:
# als_model 추천서비스
import implicit

#데이터 불러오기
spr_spr_ply_gnr_pct = spr.load_npz('data/spr_spr_ply_gnr_pct.npz') # csr형태

als_model = implicit.als.AlternatingLeastSquares(factors=50) # factors = 50
als_model.fit(spr_spr_ply_gnr_pct*40) # 40을 곱해주는건 랜덤임
user_items = spr_spr_ply_gnr_pct.T.tocsr()


  0%|          | 0/15 [00:00<?, ?it/s]

In [85]:
related = als_model.similar_items(44159, N=10)
related

[(44159, 1.0),
 (38952, 0.9993441),
 (110897, 0.99907786),
 (48088, 0.9929551),
 (25220, 0.9844763),
 (97719, 0.98434705),
 (5641, 0.9783359),
 (46828, 0.97703993),
 (79538, 0.97194),
 (3572, 0.968999)]

In [26]:
# pickle로 저장하기
import pickle
with open('data/ply2ply_als_model.pickle','wb') as fw:
    pickle.dump(als_model, fw)

# model pickle(ply2ply_als_model) 불러오기 + top10 playlist 뽑아보기

In [91]:
# model pickle(ply2ply_als_model) 불러오기
import pickle
with open('data/ply2ply_als_model.pickle','rb') as f:
    temp_model = pickle.load(f)

# top10
temp = temp_model.similar_items(10000,N=11)
temp

[(10000, 0.99999994),
 (93827, 0.9567366),
 (82698, 0.9301126),
 (11951, 0.9267973),
 (47639, 0.9232885),
 (27619, 0.92234665),
 (99261, 0.92189234),
 (7967, 0.90896934),
 (96340, 0.9086004),
 (51419, 0.9085828),
 (2909, 0.908581)]

In [30]:
# # Input 값 확인 
# display(df_train.iloc[10000])
# display(df_train.iloc[96340])

# # DF 비교 확인함
# display(df_song.iloc[df_train['songs'][10000]][['song_name','artist_name_basket','song_gn_dtl_gnr_basket']])
# display(df_song.iloc[df_train['songs'][93827]][['song_name','artist_name_basket','song_gn_dtl_gnr_basket']])

# # 가수가 비슷한 지 확인용
# temp_list_input = df_song.iloc[df_train['songs'][22]]['artist_name_basket'].tolist()
# temp_list_output = df_song.iloc[df_train['songs'][2006]]['artist_name_basket'].tolist()

# # import itertools
# # list(itertools.chain.from_iterable(my_list))

# import itertools

# temp_list_input_flat= list(itertools.chain.from_iterable(temp_list_input))
# temp_list_output_flat= list(itertools.chain.from_iterable(temp_list_output))

# print(Counter(temp_list_input_flat),'\n')
# print(Counter(temp_list_output_flat))


# 유사도가 높은 2가지 dataframe 비교
# display(df_song.iloc[df_train['songs'][1]]['artist_name_basket'].tolist())
# display(df_song.iloc[df_train['songs'][3]]['artist_name_basket'].tolist())

# # 비교한 플레이리스트 {가수 : cnt} 확인하기
# temp_list_input = df_song.iloc[df_train['songs'][1]]['artist_name_basket'].tolist()
# temp_list_output = df_song.iloc[df_train['songs'][3]]['artist_name_basket'].tolist()

# # import itertools
# # list(itertools.chain.from_iterable(my_list))

# import itertools

# temp_list_input_flat= list(itertools.chain.from_iterable(temp_list_input))
# temp_list_output_flat= list(itertools.chain.from_iterable(temp_list_output))

# print(Counter(temp_list_input_flat),'\n')
# print(Counter(temp_list_output_flat))


---